In [2]:
import glob
import os, sys

from uviz.tempest_extremes.utils import read_tempest_csv, read_tempest_ASCII
from uviz.plotting.utils import basin_bboxes#, ssh_wsp
#from uviz.utils.tools import sfc_wind_corr

from haversine import inverse_haversine, haversine_vector, haversine

import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nf4
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.lines as mlines
import matplotlib.colors as mcolors
import matplotlib.patheffects as pe
import seaborn as sns

import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
# output_folder = r"/Users/cnd5285/Library/CloudStorage/OneDrive-ThePennsylvaniaStateUniversity/Thesis_Research/github/mpas_tools/mpas_tools/datashader_tools/output_imgs"
# from PIL import Image
# import glob
# # Create the frames
# frames = []
# imgs = glob.glob(os.path.join(output_folder, "*00.png"))
# imgs.sort()
# imgs = imgs[1:]
# for i in imgs:
#     new_frame = Image.open(i)
#     frames.append(new_frame)
# # Save into a GIF file that loops forever
# duration = 150
# frames[0].save(os.path.join(output_folder, f'simulated_TC{duration}.gif'), format='GIF',
#                append_images=frames[1:],
#                save_all=True,
#                duration=duration, loop=0)
# new_frame.close()

In [ ]:
# block to calculate distance from miami
def miami_storms(df, distance=500.0, units='km'):

    miami_coords = (25.775163, -80.208615)
    miami_dist = distance #km
    df['miami_dist'] = df.apply(lambda x: haversine_vector((x.lat, x.lon), 
                                                                     miami_coords, unit=units, 
                                                                     normalize=True)[0], axis=1).round(2)
    df = df[df['miami_dist'].apply(lambda x: x <= miami_dist)].reset_index(drop=True)
    storms = df['tempest_ID'].unique()
    
    return storms

# method 1: filter down to tempest_IDs and select by tempest_IDs
# method 2: return full dataframe, only graph select few

In [ ]:
def all_calcs(dataframe, distance=500, units='km'):
    # calculates distance from Miami
    miami_coords = (25.775163, -80.208615)
    miami_dist = distance #km
    df = dataframe.copy()
    df['miami_dist'] = df.apply(lambda x: haversine_vector((x.lat, x.lon), 
                                                                     miami_coords, unit=units, 
                                                                     normalize=True)[0], axis=1).round(2)
    
    # calculates Saffir-Simpson scale
    df['ss_wsp'] = df.apply(lambda x: ssh_wsp(x.wsp, 'm/s'), axis=1)
    
    # categorizes basin
    
    
    return df

In [ ]:
#tracks_folder = r"/Users/cnd5285/Library/CloudStorage/OneDrive-ThePennsylvaniaStateUniversity/DeCiampa/model_traj/"
tracks_folder = '/gpfs/scratch/cnd5285'
te_track_files = glob.glob(os.path.join(tracks_folder, '*[!.csv]'), recursive=False)
te_track_files

In [ ]:
te_track_files[0].split('/')[-1]

In [ ]:
colnames = ['timestep', 'lon', 'lat', 'slp', 'wsp', 'sfc_phi', 'year', 'month', 'day', 'hour']
te_track_dfs = [read_tempest_ASCII(f, colnames) for f in te_track_files]
te_tracks = [all_calcs(df) for df in te_track_dfs]

In [ ]:
num_tracks = [len(track.tempest_ID.unique()) for track in te_tracks]
num_tracks = np.array(num_tracks)
total_tracks = np.sum(num_tracks)

num_miami_tracks = [len(track[track.miami_dist <= 500].tempest_ID.unique()) for track in te_tracks]
num_miami_tracks = np.array(num_miami_tracks)
tot_miami_tracks = np.sum(num_miami_tracks)

print(f'Number of unique tracks in TempestExtremes: {total_tracks} tracks.')
print(f'Number of tracks within 500 km of Miami: {tot_miami_tracks} tracks.')
print('Number of tracks that met criteria: 19 tracks.')

In [ ]:
te_track_dfs[0]

In [ ]:
te_tracks[0]

In [ ]:
min_slp_1279 = te_tracks[-1][te_tracks[-1]['tempest_ID'] == 'storm_1279']['slp'].values.min()
te_tracks[-1][(te_tracks[-1]['tempest_ID'] == 'storm_1279') & (te_tracks[-1]['slp'] == min_slp_1279)]

In [ ]:
storm_1279 = te_tracks[-1][te_tracks[-1]['tempest_ID'] == 'storm_1279'].reset_index(drop=True)
storm_1279.to_csv('../tempest_extremes/storm_1279_track.csv')

In [ ]:
def sshws_color(wsp, units='m/s'):
    category = saffir_simpson(wsp, units)
    
    if category == 'Tropical Depression':
        return '#5EBAFF'
    elif category == 'Tropical Storm':
        return '#00FAF4'
    elif category == 'Category 1':
        return '#FFF795'
    elif category == 'Category 2':
        return '#FFD821'
    elif category == 'Category 3':
        return '#FF8F20'
    elif category == 'Category 4':
        return '#FF6060'
    elif category == 'Category 5':
        return '#C464D9'
    

In [ ]:
def geog_features(ax, basin='north atlantic zoomed', resolution='10m'):
    
    lons, lats = basin_bboxes(basin)
    ax.set_extent([lons[0], lons[1], lats[0], lats[1]], crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE.with_scale(resolution), linewidth=0.5, edgecolor='#323232', zorder=3)
    #ax.add_feature(cfeature.BORDERS.with_scale(resolution), linewidth=0.5, edgecolor='#323232', zorder=3)
    ax.add_feature(cfeature.STATES.with_scale(resolution), linewidth=0.5, facecolor='#EBEBEB', edgecolor='#616161', zorder=2)
    ax.add_feature(cfeature.LAKES.with_scale(resolution), linewidth=0.5, facecolor='#e4f1fa', edgecolor='#616161', zorder=2)
    ax.add_feature(cfeature.OCEAN.with_scale(resolution), facecolor='#e4f1fa', edgecolor='face', zorder=1)

In [ ]:
def plot_sshws_segments(ax, df, figtitle=None):
    
    proj = ccrs.PlateCarree()
    for track, track_df in df.groupby('tempest_ID'):
    
        lons = track_df['lon'].values
        lats = track_df['lat'].values
        wsps = track_df['wsp'].values
        sshws_cmap = [sshws_color(x, units='m/s') for x in wsps]

        points = np.array([lons, lats]).T.reshape(-1, 1, 2)
        segments = np.concatenate([points[:-1], points[1:]], axis=1)

        lc = LineCollection(segments, colors=sshws_cmap, zorder=10, transform=proj, 
                            lw=1.25, path_effects=[pe.Stroke(linewidth=2.0, foreground='#848484'), pe.Normal()])
        ax.add_collection(lc)
    
    lw = 2.0
    lw_e = 3.0
    td = mlines.Line2D([], [], ls='-', lw=lw, label='Tropical Depression', color=sshws_color(35, 'mph'),
                       path_effects=[pe.Stroke(linewidth=lw_e, foreground='#848484'), pe.Normal()])
    ts = mlines.Line2D([], [], ls='-', lw=lw, label='Tropical Storm', color=sshws_color(50, 'mph'), 
                       path_effects=[pe.Stroke(linewidth=lw_e, foreground='#848484'), pe.Normal()])
    c1 = mlines.Line2D([], [], ls='-', lw=lw, label='Category 1', color=sshws_color(75, 'mph'), 
                       path_effects=[pe.Stroke(linewidth=lw_e, foreground='#848484'), pe.Normal()])
    c2 = mlines.Line2D([], [], ls='-', lw=lw, label='Category 2', color=sshws_color(100, 'mph'), 
                       path_effects=[pe.Stroke(linewidth=lw_e, foreground='#848484'), pe.Normal()])
    c3 = mlines.Line2D([], [], ls='-', lw=lw, label='Category 3', color=sshws_color(115, 'mph'), 
                       path_effects=[pe.Stroke(linewidth=lw_e, foreground='#848484'), pe.Normal()])
    c4 = mlines.Line2D([], [], ls='-', lw=lw, label='Category 4', color=sshws_color(135, 'mph'), 
                       path_effects=[pe.Stroke(linewidth=lw_e, foreground='#848484'), pe.Normal()])
    c5 = mlines.Line2D([], [], ls='-', lw=lw, label='Category 5', color=sshws_color(160, 'mph'), 
                       path_effects=[pe.Stroke(linewidth=lw_e, foreground='#848484'), pe.Normal()])

    l = ax.legend(handles = [c5, c4, c3, c2, c1, ts, td], loc='upper right', fontsize=14, shadow=True)
    l.set_zorder(1001)
    plt.title(figtitle, fontsize=20)
    plt.show()

In [ ]:
def plot_sshws_points(ax, df, figtitle=None, j=None, label_tracks=False):
    
    proj = ccrs.PlateCarree()
    for i, (track_ID, track_df) in enumerate(df.groupby('tempest_ID')):
        track_df = track_df.sort_values(by=['tempest_ID', 'time']).reset_index(drop=True)
        lons = track_df['lon'].values
        lats = track_df['lat'].values
        wsps = track_df['wsp'].values
        sshws_cmap = [sshws_color(x, units='m/s') for x in wsps]
        
        if j == 0:
            label_pos = [[lons[8]+1.25, lats[8]+0.2], [lons[13]+1.25, lats[13]+0.2], [lons[10]-1.25, lats[10]-0.2]]
        elif j == 1:
            label_pos = [[lons[13]-1.25, lats[13]+0.2], [lons[6]-1.25, lats[6]+0.2], [lons[8]-1.25, lats[8]-0.2], [lons[3]+1.25, lats[3]+0.2]]
        elif j == 2:
            label_pos = [[lons[13]-1.25, lats[13]-0.2], [lons[11]+1.25, lats[11]+0.2], [lons[-9]-1.25, lats[-9]-0.2], [lons[8]+1.25, lats[8]+0.2]]
        elif j == 3:
            label_pos = []
        elif j == 4:
            label_pos = [[lons[14]-1.25, lats[14]+0.2], [lons[17]+1.25, lats[17]+0.35], [lons[-10]-1.1, lats[-10]-0.35]]
        elif j == 5:
            label_pos = [[lons[11]+1.25, lats[11]+0.2]]
        elif j == 6:
            label_pos = []
        elif j == 7:
            label_pos = [[lons[4]+1.25, lats[4]+0.2], [lons[0]-1.25, lats[0]-0.2]]
        elif j == 8:
            label_pos = [[lons[10]-1.25, lats[10]+0.2], [lons[14]-1.25, lats[14]-0.2]]

        points = np.array([lons, lats]).T.reshape(-1, 1, 2)
        segments = np.concatenate([points[:-1], points[1:]], axis=1)

        lc = LineCollection(segments, colors='k', zorder=9, transform=proj, lw=0.5, ls='--')
        ax.add_collection(lc)
        ax.scatter(lons, lats, c=sshws_cmap, zorder=10, edgecolors='k', lw=0.35, s=30)
        # This labels individual tracks within model runs
        if label_tracks == True:
            ax.text(label_pos[i][0], label_pos[i][1], track_ID, transform=proj, fontsize=7.5, 
                    path_effects=[pe.Stroke(linewidth=1.75, foreground='w'), pe.Normal()], clip_on=True, ha='center', va='center')
    
    # Marker properties
    mew = 0.25     # marker edge width
    mec = 'k'      # marker edge color
    ms = 6         # marker size
    #mfc           # marker face color
    
    td = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Tropical Depression', mfc=sshws_color(35, 'mph'), color='k', lw=0.5, ls='--')
    ts = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Tropical Storm', mfc=sshws_color(50, 'mph'), color='k', lw=0.5, ls='--')
    c1 = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Category 1', mfc=sshws_color(75, 'mph'), color='k', lw=0.5, ls='--')
    c2 = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Category 2', mfc=sshws_color(100, 'mph'), color='k', lw=0.5, ls='--')
    c3 = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Category 3', mfc=sshws_color(115, 'mph'), color='k', lw=0.5, ls='--')
    c4 = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Category 4', mfc=sshws_color(135, 'mph'), color='k', lw=0.5, ls='--')
    c5 = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Category 5', mfc=sshws_color(160, 'mph'), color='k', lw=0.5, ls='--')

    l = ax.legend(handles = [c5, c4, c3, c2, c1, ts, td], loc='upper right', 
                  fontsize=10, shadow=False)
    l.set_zorder(1001)
    plt.title(figtitle)
    #plt.show()

In [ ]:
selected_tracks = [['storm_0190', 'storm_0236', 'storm_1410'],
           ['storm_0482', 'storm_0902', 'storm_1001', 'storm_1307'], 
           ['storm_0168', 'storm_0234', 'storm_0773', 'storm_1048'], None, 
           ['storm_0257', 'storm_0991', 'storm_1354'], 
           ['storm_0755'], None, 
           ['storm_0198', 'storm_0631'], 
           ['storm_0254', 'storm_1279']]

miami_coords = (25.775163, -80.208615)
florida_bbox = basin_bboxes('florida')

proj = ccrs.PlateCarree()
fig, axs = plt.subplots(3, 3, figsize=(17, 17), dpi=200, subplot_kw=dict(projection=proj))


for i, ax in enumerate(axs.ravel()):
    try:
        subset_df = te_tracks[i][(te_tracks[i].lon >= florida_bbox[0][0]-1.0) & (te_tracks[i].lon <= florida_bbox[0][1]+2.0) &\
                                 (te_tracks[i].lat >= florida_bbox[1][0]-1.0) & (te_tracks[i].lat <= florida_bbox[1][1]+2.0) &\
                                 (te_tracks[i]['tempest_ID'].isin(selected_tracks[i]))]
    except:
        continue
    title = te_track_files[i].split('/')[-1]
    geog_features(ax, basin='florida')
    plot_sshws_points(ax, subset_df, j=i, label_tracks=True)
    ax.set_title(label=title, fontsize=10, fontweight='bold')
    ax.scatter(x=miami_coords[1]+360, y=miami_coords[0], zorder=8, marker='*', c='#00FF00', s=120, edgecolors='k', lw=0.5, transform=proj)

fig.tight_layout(h_pad=-20.0)
#fig.suptitle('TempestExtremes Miami Tracks (Selected)')
fig.delaxes(axs[1][0])
fig.delaxes(axs[2][0])

plt.show()
# fig.savefig('../figs/te_plots/all_runs_selected.png', facecolor='w', transparent=False, bbox_inches='tight')
# fig.savefig('/Users/cnd5285/Library/CloudStorage/OneDrive-ThePennsylvaniaStateUniversity/DeCiampa/model_traj/figs/all_runs_selected.png', 
#             facecolor='w', transparent=False, bbox_inches='tight')

In [ ]:
selected_tracks = [['storm_0190', 'storm_0236', 'storm_1410'],
           ['storm_0482', 'storm_0902', 'storm_1001', 'storm_1307'], 
           ['storm_0168', 'storm_0234', 'storm_0773', 'storm_1048'], None, 
           ['storm_0257', 'storm_0991', 'storm_1354'], 
           ['storm_0755'], None, 
           ['storm_0198', 'storm_0631'], 
           ['storm_0254', 'storm_1279']]

i=0
subset_df = te_tracks[i][(te_tracks[i].lon >= florida_bbox[0][0]-1.0) & (te_tracks[i].lon <= florida_bbox[0][1]+2.0) &\
                         (te_tracks[i].lat >= florida_bbox[1][0]-1.0) & (te_tracks[i].lat <= florida_bbox[1][1]+2.0) &\
                         (te_tracks[i]['tempest_ID'].isin(selected_tracks[i]))]
#title = 'TempestExtremes Miami Tracks (Selected)'
title = te_track_files[i].split('/')[-1]
miami_coords = (25.775163, -80.208615)

proj = ccrs.PlateCarree()
fig, ax = plt.subplots(figsize=(12,7), dpi=200, subplot_kw=dict(projection=proj))
geog_features(ax, basin='florida')
plot_sshws_points(ax, subset_df, title, label_tracks=True, j=i)
ax.scatter(x=miami_coords[1]+360, y=miami_coords[0], zorder=8, marker='*', c='#00FF00', s=120, edgecolors='k', lw=0.5, transform=proj)
plt.show()

In [ ]:
selected_tracks = [['storm_0190', 'storm_0236', 'storm_1410'],
           ['storm_0482', 'storm_0902', 'storm_1001', 'storm_1307'], 
           ['storm_0168', 'storm_0234', 'storm_0773', 'storm_1048'], [], 
           ['storm_0257', 'storm_0991', 'storm_1354'], 
           ['storm_0755'], [], 
           ['storm_0198', 'storm_0631'], 
           ['storm_0254', 'storm_1279']]

selected_dfs = [te_tracks[i][te_tracks[i]['tempest_ID'].isin(sel_track)].reset_index(drop=True) for i, sel_track in enumerate(selected_tracks)]
for i, track_df in enumerate(selected_dfs):
    if not track_df.empty:
        track_df.to_csv(os.path.join(tracks_folder, te_track_files[i].split('/')[-1]+'_selectedtracks.csv'), index=False)

In [ ]:
florida_bbox = basin_bboxes('florida')
florida_bbox[1]

In [ ]:
florida_bbox[1][0]-1.0

In [ ]:
te_tracks[0]

In [ ]:
selected_tracks = [['storm_190', 'storm_236', 'storm_1410'],
           ['storm_482', 'storm_902', 'storm_1001', 'storm_1307'], 
           ['storm_168', 'storm_234', 'storm_773', 'storm_1048'], None, 
           ['storm_257', 'storm_991', 'storm_1354'], 
           ['storm_755'], None, 
           ['storm_198', 'storm_631'], 
           ['storm_254', 'storm_1279']]

i=0
subset_df = te_tracks[i][te_tracks[i]['tempest_ID'].isin(selected_tracks[i])]
title = 'TempestExtremes Miami Tracks (Selected)'

proj = ccrs.PlateCarree()
fig, ax = plt.subplots(figsize=(12,7), dpi=200, subplot_kw=dict(projection=proj))
geog_features(ax)
plot_sshws_segments(ax, subset_df, title)

In [ ]:
miami_tracks = te_tracks[0][te_tracks[0]['miami_dist'] <= 100].tempest_ID.unique()
subset_df = te_tracks[0][te_tracks[0]['tempest_ID'].isin(miami_tracks)]
title = 'CAM5 TempestExtremes Tracks < 100 KM of Miami (1985-2014)'

proj = ccrs.PlateCarree()
fig, ax = plt.subplots(figsize=(12,7), dpi=300, subplot_kw=dict(projection=proj))
geog_features(ax)
plot_sshws_segments(ax, subset_df, title)
fig.savefig('../figs/te_plots/te_miami_tracks.png', facecolor='w', transparent=False, bbox_inches='tight')

In [ ]:
subset_df = te_tracks[0][te_tracks[0].time.dt.year >= 2009]
title = 'CAM5 TempestExtremes Tracks (2009-2014)'

proj = ccrs.PlateCarree()
fig, ax = plt.subplots(figsize=(12,7), dpi=300, subplot_kw=dict(projection=proj))
geog_features(ax)
plot_sshws_segments(ax, subset_df, title)
fig.savefig('../figs/te_plots/te_spaghetti.png', facecolor='w', transparent=False, bbox_inches='tight')

In [ ]:
def omit_TD_TS(df):
    omit_tracks = []
    for track, track_df in df.groupby('tempest_ID'):
        if (track_df['ss_wsp'].isin(['Tropical Depression', 'Tropical Storm'])).all():
            omit_tracks.append(track)
    return df[~df['tempest_ID'].isin(omit_tracks)]

subset_df = te_tracks[0][te_tracks[0].time.dt.year >= 2009]
subsubset_df = omit_TD_TS(subset_df)
title = 'CAM5 TempestExtremes Tracks (2004-2014)' # Only TD and/or TS tracks omitted

proj = ccrs.PlateCarree()
fig, ax = plt.subplots(figsize=(12,7), dpi=200, subplot_kw=dict(projection=proj))
geog_features(ax)
plot_sshws_segments(ax, subsubset_df, title)

In [ ]:
len(omit_tracks)

In [ ]:
subsubset_df

In [ ]:
proj=ccrs.PlateCarree(globe=None)
fig, ax = plt.subplots(figsize=(12,7), dpi=200, subplot_kw=dict(projection=proj))
ax.add_feature(cfeature.COASTLINE.with_scale('10m'), linewidth=0.5, zorder=1)
ax.add_feature(cfeature.LAKES.with_scale('10m'), linewidth=0.5, facecolor='w', edgecolor='#616161', zorder=1)

explicit_cmap = {'Tropical Depression': '#5EBAFF', 
                 'Tropical Storm': '#00FAF4', 
                 'Category 1': '#FFF795', 
                 'Category 2': '#FFD821', 
                 'Category 3': '#FF8F20', 
                 'Category 4': '#FF6060', 
                 'Category 5': '#C464D9'}

g1 = sns.scatterplot(x='lon', y='lat', data=te_tracks[0][te_tracks[0]['miami_dist'] <= 500.0], edgecolor='black', linewidth=0.15, s=30,
                hue='ss_wsp', hue_order=color_order, palette=sns.color_palette(cat_colors), transform=proj, ax=ax, zorder=10)
h,l = g1.get_legend_handles_labels()
plt.legend(h[0:8],l[0:13],bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
g2 = plt.scatter(x=-80.208615, y=25.775163, marker='*', c='#00FF00', s=200, zorder=2, edgecolor='black', linewidth=0.75)
plt.show()

In [ ]:
proj=ccrs.PlateCarree(globe=None)
fig, ax = plt.subplots(figsize=(10,5), dpi=200, subplot_kw=dict(projection=proj))

ax.set_extent([270., 290., 20., 32])
ax.add_feature(cfeature.COASTLINE.with_scale('10m'), linewidth=0.5, zorder=1)
ax.add_feature(cfeature.LAKES.with_scale('10m'), linewidth=0.5, facecolor='w', edgecolor='black', zorder=1)
#ax.add_feature(cfeature.STATES, linewidth=0.5)
#ax.add_feature(cfeature.BORDERS, linewidth=0.5)

color_order = ['Tropical Depression', 'Tropical Storm', 'Category 1', 'Category 2', 'Category 3', 'Category 4', 'Category 5']
cat_colors = ['#5EBAFF', '#00FAF4', '#FFF795', '#FFD821', '#FF8F20', '#FF6060', '#C464D9']

# sns.scatterplot(x='lon', y='lat', data=te_tracks[0][te_tracks[0]['miami_dist'] <= 500.0], 
#                 hue='ss_wsp', hue_order=color_order, palette=sns.color_palette(cat_colors), transform=proj, ax=ax)
g = sns.lineplot(x='lon', y='lat', data=te_tracks[0][te_tracks[0]['miami_dist'] <= 500.0], markers=True, style='tempest_ID',
                hue='ss_wsp', hue_order=color_order, palette=sns.color_palette(cat_colors), transform=proj, ax=ax, zorder=10)
h,l = g.get_legend_handles_labels()
plt.legend(h[0:8],l[0:13],bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show(g)
#plt.show()

In [ ]:
proj=ccrs.PlateCarree(globe=None)
fig, ax = plt.subplots(figsize=(10,5), dpi=200, subplot_kw=dict(projection=proj))

ax.set_extent([260., 358., 18., 40])
ax.add_feature(cfeature.COASTLINE.with_scale('10m'), linewidth=0.5, zorder=1)
ax.add_feature(cfeature.LAKES.with_scale('10m'), linewidth=0.5, facecolor='w', edgecolor='black', zorder=1)
#ax.add_feature(cfeature.STATES, linewidth=0.5)
#ax.add_feature(cfeature.BORDERS, linewidth=0.5)

color_order = ['Tropical Depression', 'Tropical Storm', 'Category 1', 'Category 2', 'Category 3', 'Category 4', 'Category 5']
cat_colors = ['#5EBAFF', '#00FAF4', '#FFF795', '#FFD821', '#FF8F20', '#FF6060', '#C464D9']
marker_style = dict(color='tab:blue', linestyle=':', marker=np.repeat('o', 33),
                    markersize=15)

# sns.scatterplot(x='lon', y='lat', data=te_tracks[0][te_tracks[0]['miami_dist'] <= 500.0], 
#                 hue='ss_wsp', hue_order=color_order, palette=sns.color_palette(cat_colors), transform=proj, ax=ax)
g = sns.lineplot(x='lon', y='lat', data=te_tracks[0][(te_tracks[0].lat>=18) & (te_tracks[0].lat<=60) &\
                                                     (te_tracks[0].lon>=260) & (te_tracks[0].lon<=360) &\
                                                     (te_tracks[0].time.dt.year >= 2012)], 
                 markers=marker_style, hue='ss_wsp', hue_order=color_order, palette=sns.color_palette(cat_colors), 
                 style='tempest_ID', dashes=False, transform=proj, ax=ax, zorder=10)
h,l = g.get_legend_handles_labels()
plt.legend(h[0:8],l[0:13],bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., title='Saffir Simpson Wind Scale')
plt.title('TempestExtremes Spaghetti Plot')
plt.show(g)
#plt.show()

In [ ]:
?plt.legend

In [ ]:
sns.set()

In [ ]:
os.path.isdir('../figs/te_plots', te_track_files[0].split('/')[-1]))

In [ ]:
os.path.join('../figs/te_plots', te_track_files[0].split('/')[-1]+'.png')

In [ ]:
def compare_te_tracks(df, file_name, savefig=False):
    proj=ccrs.PlateCarree(globe=None)
    color_order = ['Tropical Depression', 'Tropical Storm', 'Category 1', 'Category 2', 'Category 3', 'Category 4', 'Category 5']
    cat_colors = ['#5EBAFF', '#00FAF4', '#FFF795', '#FFD821', '#FF8F20', '#FF6060', '#C464D9']

    g = sns.FacetGrid(df[df['miami_dist'] <= 500.0], col="tempest_ID", hue='ss_wsp',
                      col_wrap=8, height=3.5, hue_order=color_order, palette=sns.color_palette(cat_colors),
                      subplot_kws=dict(projection=proj))
    #g3.map_dataframe(sns.lineplot, x='lon', y='lat')
    g.map_dataframe(sns.scatterplot, x='lon', y='lat', zorder=10, s=20, edgecolor='black', linewidth=0.5)
    g.map(plt.scatter, x=-80.208615, y=25.775163, marker='*', s=50, zorder=2, facecolor='#00FF00', edgecolor='black', linewidth=0.75)

    title = file_name.split('/')[-1]
    g.fig.suptitle(title, fontsize=16, fontweight='bold')
    g.figure.subplots_adjust(top=.96)
    g.set_titles(size=12)

    for ax in g.axes.ravel():
        #ax.set_extent([270., 290., 20., 32])
        ax.add_feature(cfeature.COASTLINE.with_scale('10m'), linewidth=0.5, zorder=1)
        ax.add_feature(cfeature.LAKES.with_scale('10m'), linewidth=0.5, facecolor='w', edgecolor='black', zorder=1)
    
    if savefig == True:
        plt.savefig(os.path.join('../figs/te_plots', title+'.png'), dpi=300)

In [ ]:
def select_te_tracks(df, stracks, file_name, savefig=False):
    df = df[df['tempest_ID'].isin(stracks)]
    proj=ccrs.PlateCarree(globe=None)
    color_order = ['Tropical Depression', 'Tropical Storm', 'Category 1', 'Category 2', 'Category 3', 'Category 4', 'Category 5']
    cat_colors = ['#5EBAFF', '#00FAF4', '#FFF795', '#FFD821', '#FF8F20', '#FF6060', '#C464D9']

    g = sns.FacetGrid(df, col="tempest_ID", hue='ss_wsp', col_wrap=4,
                      height=3.5, hue_order=color_order, palette=sns.color_palette(cat_colors),
                      subplot_kws=dict(projection=proj))
    #g3.map_dataframe(sns.lineplot, x='lon', y='lat')
    g.map_dataframe(sns.scatterplot, x='lon', y='lat', zorder=10, s=20, edgecolor='black', linewidth=0.5)
    g.map(plt.scatter, x=-80.208615, y=25.775163, marker='*', s=50, zorder=2, facecolor='#00FF00', edgecolor='black', linewidth=0.75)

    title = file_name.split('/')[-1]
    g.fig.suptitle(title, fontsize=16, fontweight='bold')
    g.figure.subplots_adjust(top=.8)
    g.set_titles(size=12)

    for ax in g.axes.ravel():
        ax.set_extent([270.5, 286., 19., 32])
        ax.add_feature(cfeature.COASTLINE.with_scale('10m'), linewidth=0.5, zorder=1)
        ax.add_feature(cfeature.LAKES.with_scale('10m'), linewidth=0.5, facecolor='w', edgecolor='black', zorder=1)
    
    if savefig == True:
        plt.savefig(os.path.join('../figs/te_plots', title+'_selected.png'), dpi=300)

In [ ]:
selected_tracks = [['storm_190', 'storm_236', 'storm_1410'],
           ['storm_482', 'storm_902', 'storm_1001', 'storm_1307'], 
           ['storm_168', 'storm_234', 'storm_773', 'storm_1048'], None, 
           ['storm_257', 'storm_991', 'storm_1354'], 
           ['storm_755'], None, 
           ['storm_198', 'storm_631'], 
           ['storm_254', 'storm_1279']]

[select_te_tracks(te_tracks[i], selected_tracks[i], f, savefig=True) for i, 
 f in enumerate(te_track_files) if selected_tracks[i] != None]

In [ ]:
[compare_te_tracks(te_tracks[i], f, savefig=True) for i, f in enumerate(te_track_files)]

In [ ]:
compare_te_tracks(te_tracks[0], te_track_files[0])

"""
Selected tracks - tracks[0]:
storm_190 - has cat 4 hit directly on Miami
storm_236 - has cat 4 (high end?) hit directly on Ft. Lauderdale?
storm_1410 - has cat 3 Keys landfall and follows west coast up (storm surge risk to Miami)

Selected tracks - tracks[1]:
storm_482 - has cat 3 second landfall on Miami, comes up the coast weird (SW-NE, surge risk)
storm_902 - weak TS/TD hangs around Miami for ~24 hours (flood risk)
storm_1001 - cat 4 Everglades landfall (surge risk, brown ocean effect)
storm_1307 - direct Miami landfall, very normal track (temporally and spatially) curved up center of Florida, cat 4-5 landfall

Selected tracks - tracks[2]:
storm_168 - cat 4 south keys landfall (somewhat surge risk)
storm_234 - cat 3 keys landfall (same as tracks[0][storm_1410])
storm_773 - same as storm_234 but further away
storm_1408 - cat 5 landfall West Palm Beach (~1 hour north of Miami)

Selected tracks - tracks[3]: none

Selected tracks - tracks[4]:
storm_257 - same as (tracks[0][storm_236])
storm_991 - cat 4 direct landfall, very normal track across Florida into Gulf
storm_1048 - cat 5 direct landfall Key Largo, 2nd landfall in Everglades (huge surge risk, brown ocean effect)

Selected tracks - tracks[5]:
storm_755 - meanders as a cat 2-3 over Cypress Nat'l Forest for ~48 hours (flood/inundation risk depending on size)

tracks[6]: none

tracks[7]:
storm_198 - cat 5 landfall in Ft. Lauderdale?
storm_631 - meandering TS across S. Florida for ~54 hours

tracks[8]:
storm_254 - TS/TD meandering across Miami for ~24 hours
storm_1279 - cat 4/5 landfall just south of Miami (huge surge risk, right front quadrant risk, tornadic risk)


"""

In [ ]:
compare_te_tracks(te_tracks[1], te_track_files[1])

In [ ]:
compare_te_tracks(te_tracks[2], te_track_files[2])

In [ ]:
compare_te_tracks(te_tracks[3], te_track_files[3])

In [ ]:
compare_te_tracks(te_tracks[4], te_track_files[4])

In [ ]:
compare_te_tracks(te_tracks[5], te_track_files[5])

In [ ]:
compare_te_tracks(te_tracks[6], te_track_files[6])

In [ ]:
compare_te_tracks(te_tracks[7], te_track_files[7])

In [ ]:
compare_te_tracks(te_tracks[8], te_track_files[8])

In [ ]:
# block to calculate translation speed (needs to be first before dropping due to miami_dist)
test_df = te_track_dfs[0].copy()
test_df['tempest_ID'].unique()
#[test_df[test_df == storm].apply(lambda x: haversine_vector((x.lat, x.lon), (x.lat, *x.lon), unit='mi', 
                                                                 normalize=True)[0], axis=1).round(4) for storm in test_df['tempest_ID'].unique()]

In [ ]:
# block to calculate translation speed (needs to be first before dropping due to miami_dist)
test_df['t_speed'] = test_df.rolling(2, on='tempest_ID', closed='both').apply(lambda x: haversine_vector(x.lat)

In [ ]:
test_df.groupby(['tempest_ID'])['lon'].shift(1)

In [ ]:
#test_df['ts1'] = test_df.groupby('tempest_ID', as_index=False)
test_df.groupby('tempest_ID').lon

In [ ]:
test_df

In [ ]:
test_df['t_speed'] = test_df.apply(lambda x: haversine_vector((x.lat, x.lon), (*x.lat, *x.lon), unit='mi', 
                                                                 normalize=True)[0], axis=1).round(4)
test_df

In [ ]:

test_df['t_speed'] = test_df.groupby('tempest_ID').\
apply(lambda x: haversine_vector([x.lat, x.lon], 
                                 [x.lat.shift().rolling(1), x.lon.shift().rolling(1)], unit='mi')).reset_index(drop=True)

In [ ]:
test_df = te_track_dfs[0].copy()
# block to calculate distance from miami
miami_coords = (25.775163, -80.208615)
miami_dist = 500.0 #km
test_df['miami_dist'] = test_df.apply(lambda x: haversine_vector((x.lat, x.lon), 
                                                                 miami_coords, unit='km', 
                                                                 normalize=True)[0], axis=1).round(4)
#test_df = test_df[test_df['miami_dist'].apply(lambda x: x <= miami_dist)].reset_index(drop=True)
test_df

In [ ]:
# block to calculate saffir-simpson
test_df['ss_wsp'] = test_df.apply(lambda x: saffir_simpson(x.wsp, 'm/s'), axis=1)
test_df

In [ ]:
# List of storm ID's for cat 5's to check
test_df[test_df['ss_wsp'] == 'Category 5'].tempest_ID.unique()

In [ ]:
test_df[test_df['ss_wsp'] == 'Category 5']

In [ ]:
# block to calculate translation speed
